In [25]:
! pip install beautifulsoup4

In [26]:
! pip install selenium

In [27]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import pytest
import time

In [28]:
# We open Chrome browser in incognito mode
service = ChromeService(executable_path=ChromeDriverManager().install())

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome(service=service, chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Mikel\.wdm\drivers\chromedriver\win32\99.0.4844.51]
<ipython-input-28-d681e9ce9338>:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=service, chrome_options=chrome_options)


In [29]:
# Link to the desired page to be scraped
driver.get('https://foxhugh.com/interview-questions/list-of-interview-questions/');

# Save all the content in the "soup" variable
content = driver.page_source
soup = BeautifulSoup(content)

#List to store de questions
questions = [] 

#The content we want is in a "ol" type layer 
new = soup.ol

#Get the content
hijos=new.findAll('li')

#Iterate the content 1 by 1
for child in hijos:
    print(child.string)
    questions.append(child.string)

#Save all the questions in a CSV
df = pd.DataFrame({'Interviewer Questions':questions}) 
df.to_csv('Interviewer_Questions.csv', index=False, encoding='utf-8')


A natural disaster occurs in Manhattan. What impact does this have on markets?
Add up all the numbers between 0-100.
Are there any particular companies that interest you?
Are there aspects to your job that are repetitious?
Are you a Coronation Street or EastEnders fan?
Are you a goal-oriented person?
Are you a leader or a follower?
Are you a lone wolf?
Are you a team player?
Are you better with data or people?
Are you comfortable with a lack of deadlines?
Are you comfortable with strict deadlines?
Are you currently taking or enrolled to take any job related educational courses?
Are you currently under any employment contract obligation form current or previous employers?
Are you good at accepting delegation?
Are you good at delegating tasks?
Are you hoping for advancement within the company?
Are you looking for a permanent or temporary position at the company?
Are you the type of student for whom conducting independent research has been a positive experience?
Are you willing to make sa

In [30]:
from selenium.webdriver.common.by import By
import pyperclip

#Now lets go to the google traductor page
driver.get('https://translate.google.es/?hl=es&sl=en&tl=es&op=translate');
time.sleep(5) # Let the user actually see something!

In [31]:
#Accept google cookies
driver.find_element(By.XPATH, "//span[normalize-space()='Acepto']").click()

In [32]:
#We will iterate over all the saved questions and translate them all, we have 1003
print(len(questions))

1003


In [33]:
print(questions)



['A natural disaster occurs in Manhattan. What impact does this have on markets?', 'Add up all the numbers between 0-100.', 'Are there any particular companies that interest you?', 'Are there aspects to your job that are repetitious?', 'Are you a Coronation Street or EastEnders fan?', 'Are you a goal-oriented person?', 'Are you a leader or a follower?', 'Are you a lone wolf?', 'Are you a team player?', 'Are you better with data or people?', 'Are you comfortable with a lack of deadlines?', 'Are you comfortable with strict deadlines?', 'Are you currently taking or enrolled to take any job related educational courses?', 'Are you currently under any employment contract obligation form current or previous employers?', 'Are you good at accepting delegation?', 'Are you good at delegating tasks?', 'Are you hoping for advancement within the company?', 'Are you looking for a permanent or temporary position at the company?', 'Are you the type of student for whom conducting independent research ha

In [34]:
# Global list to save the translated questions
questions_translated = []

In [35]:
len(questions)

1003

In [36]:
i=0
stop=0

def questions_traductor(index):
    i=index
    while i < len(questions):
        #if the program fails, stop will save the index and it will restart at the point where failed
        stop=i
        
        # Print the position and question to see if everything goes well
        print(i)
        print(questions[i])
        
        # If the question actually exists (errors or blank lines could be imported in the scrapping)
        if questions[i] is not None:
        
            # Get the search_box to send the question
            try:
                search_box = driver.find_element(By.CLASS_NAME, "er8xn")
            except:
                questions_traductor(stop)

            # Questions sended to the translator
            search_box.send_keys(questions[i])

            # Sleep needed to let the traductor respond
            time.sleep(2)

            # find the copy translation button and click on it
            try:
                driver.find_element(By.XPATH, "//button[@aria-label='Copiar traducción']//div[@class='VfPpkd-Bz112c-RLmnJb']").click()
            except:
                questions_traductor(stop)

            time.sleep(2)

            # paste the translation saved in the clipboard to a variable
            translation = []
            translation = pyperclip.paste()
            print(translation)

            # Append the translation
            questions_translated.append(translation)

            # Clear the search box to keep translating
            search_box.clear()
        
        # Next question, doesnt matter if the question before existed or not
        i=i+1

In [37]:
questions_traductor(stop)

0
A natural disaster occurs in Manhattan. What impact does this have on markets?
Se produce un desastre natural en Manhattan. ¿Qué impacto tiene esto en los mercados?
1
Add up all the numbers between 0-100.
Suma todos los números entre 0-100.
2
Are there any particular companies that interest you?
¿Hay alguna empresa en particular que te interese?
3
Are there aspects to your job that are repetitious?
¿Hay aspectos de su trabajo que son repetitivos?
4
Are you a Coronation Street or EastEnders fan?
¿Eres una calle de coronación o fanático de Eastenders?
5
Are you a goal-oriented person?
¿Eres una persona orientada a objetivos?
6
Are you a leader or a follower?
¿Eres un líder o un seguidor?
7
Are you a lone wolf?
¿Eres un lobo solitario?
8
Are you a team player?
¿Eres un jugador de equipo?
9
Are you better with data or people?
¿Eres mejor con datos o personas?
10
Are you comfortable with a lack of deadlines?
¿Estás cómodo con la falta de plazos?
11
Are you comfortable with strict deadline

KeyboardInterrupt: 

In [ ]:
#Save all the questions in a CSV
df = pd.DataFrame({'Interviewer Questions Translated':questions_translated}) 
df.to_csv('Interviewer_Questions_Translated_v1.csv', index=False, encoding='utf-8')

In [ ]:
len(questions_translated)

In [ ]:
print(questions_translated)

In [ ]:
#Close the driver
driver.quit()